In [1]:
import pandas as pd
import numpy as np
import xlsxwriter
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
df = pd.read_csv('C:\Documents\Premi_ClubQ8.csv', sep = ';', decimal=",")


df[['REQUEST_DATE','REQUEST_TIME']] = df.DATA_OPERAZIONE.str.split(" ",expand=True,)

df = df.drop(columns="DATA_OPERAZIONE")

df.columns = ['CUSTOMER_CODE', 'REQUEST_PLACE', 'CATEGORY', 'MATERIAL_GROUPING', 'PRIZE_DESCRIPTION', 'POINTS_REQUIRED', 'CUSTOMER_CONTRIBUTION', 'DATE', 'TIME']
df.dtypes

df['CUSTOMER_CONTRIBUTION'] = df['CUSTOMER_CONTRIBUTION'].astype(float)

df
#df['CUSTOMER_CONTRIBUTION'] = df['CUSTOMER_CONTRIBUTION'].round(decimals = 1)



#df['CUSTOMER_CONTRIBUTION'] = df['CUSTOMER_CONTRIBUTION'].fillna(0.0)

,CUSTOMER_CODE,REQUEST_PLACE,CATEGORY,MATERIAL_GROUPING,PRIZE_DESCRIPTION,POINTS_REQUIRED,CUSTOMER_CONTRIBUTION,DATE,TIME
0,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:16:21
1,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:17:30
2,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:18:49
3,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:19:42
4,7071161603014051338,PUNTO VENDITA,PREMIO FISICO,PER TE,PRENOTAZIONE PORTA CARTE CREDITO SPARCO,450.0,0.0,2020-09-01,00:34:24
...,...,...,...,...,...,...,...,...,...
501444,7071161603014068530,PUNTO VENDITA,PREMIO FISICO,ELETTRODOMESTICI,PRENOTAZIONE PHILIPS RASOIO,2395.0,0.0,2021-08-31,23:14:00
501445,7071161603670271808,PUNTO VENDITA,PREMIO FISICO,MODA,PRENOTAZIONE OROLOGIO SECTOR,3500.0,30.0,2021-08-31,23:14:59
501446,7071161603014068530,PUNTO VENDITA,PREMIO FISICO,ELETTRONICA,PRENOTAZIONE AVVITATORE,4200.0,15.0,2021-08-31,23:15:02
501447,7071161603007816325,PUNTO VENDITA,PREMIO FISICO,CASA,PRENOTAZIONE PORTABIANCHERIA,4175.0,0.0,2021-08-31,23:24:29


In [3]:
def summarize_na(df: pd.DataFrame) -> pd.DataFrame:
    nan_count = df.isna().sum()
    return pd.DataFrame({'nan_count': nan_count, 
                         'nan_pct': nan_count / len(df) * 100
                         }
                        )[nan_count > 0]

In [4]:
df_nan_sum = summarize_na(df)
df_nan_sum

,nan_count,nan_pct
POINTS_REQUIRED,229,0.045668
CUSTOMER_CONTRIBUTION,37895,7.557100


In [5]:
df['CATEGORY'].unique()

array(['PREMIO FISICO', 'BUONO SCONTO CARBURANTE'], dtype=object)

In [6]:
tab = pd.crosstab(index=df['REQUEST_PLACE'], columns='proportion')
tab/tab.sum()

col_0,proportion
REQUEST_PLACE,
ONLINE,0.064593
PUNTO VENDITA,0.935407


In [7]:
tab = pd.crosstab(index=df['CATEGORY'], columns='proportion')
tab/tab.sum()

col_0,proportion
CATEGORY,
BUONO SCONTO CARBURANTE,0.543744
PREMIO FISICO,0.456256


In [8]:
tab = pd.crosstab(index=df['MATERIAL_GROUPING'], columns='proportion')
tab/tab.sum()

col_0,proportion
MATERIAL_GROUPING,
AMICI A 4 ZAMPE,0.003099
BAMBINI,0.003488
BELLEZZA & BENESSERE,0.001392
BUONO SCONTO CARBURANTE,0.543744
CASA,0.036951
ELETTRODOMESTICI,0.034877
ELETTRONICA,0.049704
GUARDA & PRENOTA,0.002724
MODA,0.014432


In [9]:
tab = pd.crosstab(index=df['PRIZE_DESCRIPTION'], columns='proportion')
tab/tab.sum()

col_0,proportion
PRIZE_DESCRIPTION,
ANNULLO BUONO SCONTO CARB 4E 1290 PUNTI,0.000086
ANNULLO BUONO SCONTO CARB 4E 2580 PUNTI,0.000108
ANNULLO BUONO SCONTO CARB 4E 430 PUNTI,0.000044
ANNULLO BUONO SCONTO CARB 4E 860 PUNTI,0.000028
ANNULLO CONSEGNA MASCHERINA TOMBOLINI,0.000012
...,...
PRENOTAZIONE ZAINO BATMAN,0.000070
PRENOTAZIONE ZAINO CABIN ZERO,0.000598
PRENOTAZIONE ZAINO CABINZERO NERO,0.000718


In [10]:
tab = pd.crosstab(index=df['POINTS_REQUIRED'], columns='proportion')
tab/tab.sum()

col_0,proportion
POINTS_REQUIRED,
0.0,0.000148
79.0,0.000483
100.0,0.009846
300.0,0.000271
350.0,0.037975
...,...
18295.0,0.000118
18500.0,0.000012
20545.0,0.000120


In [11]:
tab = pd.crosstab(index=df['CUSTOMER_CONTRIBUTION'], columns='proportion')
tab/tab.sum()

col_0,proportion
CUSTOMER_CONTRIBUTION,
0.0,0.840355
1.0,0.006079
2.0,0.015666
3.0,0.009919
4.0,0.005747
5.0,0.029013
6.0,0.013642
7.0,0.003454
8.0,0.014693


In [12]:
tab = pd.crosstab(index=df['DATE'], columns='proportion')
tab/tab.sum()

col_0,proportion
DATE,
2020-09-01,0.003235
2020-09-02,0.003253
2020-09-03,0.003235
2020-09-04,0.003635
2020-09-05,0.003171
...,...
2021-08-27,0.002937
2021-08-28,0.002503
2021-08-29,0.000730


In [13]:
df2 = pd.read_csv('C:\Documents\Anagrafica_ClubQ8.csv', sep = ';', decimal=",")

df2['DATE_JOINED'] = df2['DATA_BATTESIMO']

df2 = df2.drop(columns="DATA_BATTESIMO")

df2.columns = ['CUSTOMER_CODE', 'SEX', 'DOB', 'REGION', 'PROVINCE', 'CITY', 'TYPE_OF_CARD' ,'TOTAL_POINTS', 'DATE_JOINED']

df2

,CUSTOMER_CODE,SEX,DOB,REGION,PROVINCE,CITY,TYPE_OF_CARD,TOTAL_POINTS,DATE_JOINED
0,7071161603018922195,M,1951-09-14 00:00:00,LAZIO,VITERBO,SUTRI,STARQ8_FISICA,511.0,2015-01-08 12:32:58
1,7071161603005992565,F,1988-09-15 00:00:00,NaN,NaN,NaN,STARQ8_FISICA,379.0,2020-07-10 12:22:35
2,7071161603025867243,F,1943-09-09 00:00:00,LAZIO,ROMA,ROMA,STARQ8_FISICA,108.0,2020-07-10 18:07:14
3,7071161603021997861,M,1974-03-01 00:00:00,SICILIA,MESSINA,MESSINA,STARQ8_FISICA,100.0,2020-07-10 18:08:21
4,7071161603025554049,M,1963-11-22 00:00:00,CAMPANIA,SALERNO,EBOLI,STARQ8_FISICA,284.0,2020-07-10 18:09:27
...,...,...,...,...,...,...,...,...,...
1656132,7071161603672630225,F,1962-06-22 00:00:00,EMILIA ROMAGNA,RIMINI,SAN GIOVANNI IN MARIGNANO,STARQ8_FISICA,2741.0,2019-11-27 18:06:31
1656133,7071161603673015798,M,1967-03-19 00:00:00,LAZIO,ROMA,ROMA,STARQ8_FISICA,8996.0,2019-03-21 08:34:30
1656134,7071161603024035909,M,1987-03-18 00:00:00,SICILIA,PALERMO,PALERMO,STARQ8_FISICA,424.0,2021-01-19 09:07:37
1656135,7071161603601002132,F,1984-09-20 00:00:00,LOMBARDIA,BRESCIA,ISEO,STARQ8_VIRTUALE,740.0,2020-12-15 22:57:21


In [15]:
df = pd.merge(df, df2, how = 'left')
df

,CUSTOMER_CODE,REQUEST_PLACE,CATEGORY,MATERIAL_GROUPING,PRIZE_DESCRIPTION,POINTS_REQUIRED,CUSTOMER_CONTRIBUTION,DATE,TIME,SEX,DOB,REGION,PROVINCE,CITY,TYPE_OF_CARD,TOTAL_POINTS,DATE_JOINED
0,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:16:21,M,1978-09-28 00:00:00,SICILIA,MESSINA,MESSINA,STARQ8_FISICA,3997.0,2013-05-13 09:16:29
1,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:17:30,M,1978-09-28 00:00:00,SICILIA,MESSINA,MESSINA,STARQ8_FISICA,3997.0,2013-05-13 09:16:29
2,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:18:49,M,1978-09-28 00:00:00,SICILIA,MESSINA,MESSINA,STARQ8_FISICA,3997.0,2013-05-13 09:16:29
3,7071161603670014604,ONLINE,PREMIO FISICO,PER LA TUA CASA,CONSEGNA CODICE GIFT CARDS 20E AMAZON,1800.0,NaN,2020-09-01,00:19:42,M,1978-09-28 00:00:00,SICILIA,MESSINA,MESSINA,STARQ8_FISICA,3997.0,2013-05-13 09:16:29
4,7071161603014051338,PUNTO VENDITA,PREMIO FISICO,PER TE,PRENOTAZIONE PORTA CARTE CREDITO SPARCO,450.0,0.0,2020-09-01,00:34:24,M,1967-09-02 00:00:00,SICILIA,PALERMO,CEFALU',STARQ8_FISICA,5279.0,2001-10-05 11:34:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501444,7071161603014068530,PUNTO VENDITA,PREMIO FISICO,ELETTRODOMESTICI,PRENOTAZIONE PHILIPS RASOIO,2395.0,0.0,2021-08-31,23:14:00,M,1975-07-11 00:00:00,NaN,NaN,NaN,STARQ8_FISICA,111.0,2017-07-04 13:00:40
501445,7071161603670271808,PUNTO VENDITA,PREMIO FISICO,MODA,PRENOTAZIONE OROLOGIO SECTOR,3500.0,30.0,2021-08-31,23:14:59,M,1977-11-29 00:00:00,CAMPANIA,NAPOLI,NAPOLI,STARQ8_FISICA,9590.0,2012-11-08 10:28:34
501446,7071161603014068530,PUNTO VENDITA,PREMIO FISICO,ELETTRONICA,PRENOTAZIONE AVVITATORE,4200.0,15.0,2021-08-31,23:15:02,M,1975-07-11 00:00:00,NaN,NaN,NaN,STARQ8_FISICA,111.0,2017-07-04 13:00:40
501447,7071161603007816325,PUNTO VENDITA,PREMIO FISICO,CASA,PRENOTAZIONE PORTABIANCHERIA,4175.0,0.0,2021-08-31,23:24:29,M,1967-11-13 00:00:00,MARCHE,ANCONA,ANCONA,STARQ8_FISICA,1041.0,2015-08-28 15:56:15
